In [1]:
# Konlpy의 Okt(twitter)로 토크나이징하기
from konlpy.corpus import kolaw
from konlpy.tag import *
import konlpy

openkoreantext = Okt()

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

# tokenizer : 문장에서 색인어 추출을 위해 명사,동사,알파벳,숫자 정도의 단어만 뽑아서 normalization, stemming 처리하도록 함
def tokenizer(raw, pos=["Noun","Alpha","Verb","Number"], stopword=[]):
    return [
        word for word, tag in openkoreantext.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

In [2]:
info_naver = pd.read_csv("data_Juno/webtoon_naver_info_by_genres.csv")
info_naver.tail(1)

,unique_id,title,author,total_score,main_link,image_link,update_date,genre,pay,total_like,intro,total_episodes,mean_score,total_comments,mean_like
744,347685,슈퍼트리오 시즌2,황미나,7.36,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2012.06.04,story/comic/drama,1,29,대한민국 명작 만화 초대선.\r\r\n살아 있는 전설 황미나 작가의 명작 “슈퍼트리...,6,7.35,1374,3.4


In [3]:
info_naver['text'] = info_naver['genre'] + info_naver['intro']
info_naver.tail(1)

,unique_id,title,author,total_score,main_link,image_link,update_date,genre,pay,total_like,intro,total_episodes,mean_score,total_comments,mean_like,text
744,347685,슈퍼트리오 시즌2,황미나,7.36,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2012.06.04,story/comic/drama,1,29,대한민국 명작 만화 초대선.\r\r\n살아 있는 전설 황미나 작가의 명작 “슈퍼트리...,6,7.35,1374,3.4,story/comic/drama대한민국 명작 만화 초대선.\r\r\n살아 있는 전설...


In [4]:
vectorize = TfidfVectorizer(
    tokenizer=tokenizer,
    min_df=1,
    sublinear_tf=True    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음
).fit(info_naver['intro'])

In [5]:
X = vectorize.transform(info_naver['intro'])
# 문장에서 뽑아낸 feature 들의 배열
features = vectorize.get_feature_names()

# search 문장 벡터
srch_vector = vectorize.transform([
    "트레이스, 트러블, 인간.. 그들의 생존을 건 싸움",
])

# print(srch_vector.shape)
# srch_vector.toarray()
# features


# linear_kernel은 두 벡터의 dot product이다.
print(srch_vector.shape, X.shape)
cosine_similar = linear_kernel(srch_vector,X).flatten()
# 위 코드는 아래와 같은 역할을 한다
# cosine_similar = (srch_vector*X.T).toarray().flatten()

# 유사한 rawdata index
sim_rank_idx = cosine_similar.argsort()[::-1]
# print(sim_rank_idx)

for i in sim_rank_idx:
    if cosine_similar[i] > 0 :
        print('title : {} / Intro : {} / score : {}'.format(info_naver.loc[i, 'title'],info_naver['intro'].values[i],cosine_similar[i]))

(1, 2902) (745, 2902)
title : 심연의 하늘 시즌4 / Intro : 재난으로 파괴된 대한민국. 생존을 위한 인간들의 잔혹한 축제가 벌어진다. / score : 0.37692360480737586

생존을 위한 처절한 몸부림. / score : 0.2720590885528513

쫓고 쫓기는 그들의 스릴 넘치는 이야기, '기사도' / score : 0.210950924679459

세계의 운명을 건 싸움이 지금 시작된다 / score : 0.20478793778183313
title : 하나의 하루 / Intro : 인간소녀를 향한 늑대인간의 사랑 / score : 0.20472670909652188
title : 내일은 럭키곰스타 / Intro : 삼류라 불리는 인형이지만, 인간보다 더 인간적인 그들이 펼치는 배꼽빠지게 슬픈이야기 / score : 0.19827424576658592
title : 팀 피닉스 / Intro : 선과 악이 뒤섞인 세상초능력 범죄자와 초능력 경찰의 치열한 싸움이 시작된다 / score : 0.1923357194974853
title : 악의는 없다 / Intro : 시골 마을, 한 남자가 죽었다.그 죽음의 진실을 파헤치려는 이와 덮으려는 자의 싸움. / score : 0.1913066619459935

이제 적도 아군도 없는 싸움이 펼쳐진다. / score : 0.18537479397572185
title : 용이산다 / Intro : 드래곤이든 인간이든 그저 평범한 판타지 일상물.새로운 용(?)물 등장! / score : 0.1575816938574721
title : 신도림 / Intro : 세상이 무너지고 세워진 지하도시 신도림그곳에 들어가지 못하고 생존한 소년들이 지하도시의 비밀을 파헤친다 / score : 0.15583400135848777


살아남은 자들의 숨막히는 생존기 / score : 0.15382229333970626
title : 환생동물학교 / Intro : 동물들이 인간으로 환생하기 

In [57]:
test = info_naver.copy()
test['similarity'] = cosine_similar
test[test.title == "심연의 하늘 시즌4"]

,unique_id,title,author,total_score,main_link,image_link,update_date,genre,pay,total_like,intro,total_episodes,mean_score,total_comments,mean_like,similarity
646,679547,심연의 하늘 시즌4,윤인완/김선희,7.01,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2016.06.20,story/thrill,1,58706,재난으로 파괴된 대한민국. 생존을 위한 인간들의 잔혹한 축제가 벌어진다.,3,5.39,29099,12032.67,0.376924


In [8]:
info_naver.tail(1)

,unique_id,title,author,total_score,main_link,image_link,update_date,genre,pay,total_like,intro,total_episodes,mean_score,total_comments,mean_like
744,347685,슈퍼트리오 시즌2,황미나,7.36,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2012.06.04,story/comic/drama,1,29,대한민국 명작 만화 초대선.\r\r\n살아 있는 전설 황미나 작가의 명작 “슈퍼트리...,6,7.35,1374,3.4


### 1st Modeling Process
- feature select : total_like, mean_score, mean_like, total_comments(평균으로 쓸지 고려)
- feature normalization : 0~1사의 값으로 정규화할 것인가 or 평균 0, 표준편차 1로 만들 것인가.
- 재밌게 본 웹툰 선택시 유사도 측정 --> feature 추가

In [11]:
len(info_naver[info_naver['title'] == '슈퍼트리오 시즌2'])

1

In [1]:
from konlpy.corpus import kolaw
from konlpy.tag import *
import konlpy

openkoreantext = Okt()

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

from sklearn.preprocessing import minmax_scale

In [2]:
# min-max scale
def make_scale(df):
    """
    웹툰 수치를 0~1사이로 통일하기위한 함수
    """
    df = df[['mean_like','total_comments','mean_score']]
    scaled_df = pd.DataFrame()
    for column in df.columns:
        scaled_df[column] = minmax_scale(df[column])
    return scaled_df

# 토크나이저
def tokenizer(raw, pos=["Noun","Alpha","Verb","Number"], stopword=[]):
    return [
        word for word, tag in openkoreantext.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

# 해당 웹툰의 소개글 찾기
def find_intro(df, webtoon_name):
    return df[df.title == webtoon_name].intro # 여기서 나온 intro가 get_similarity의 parameter로 간다.

vectorize = TfidfVectorizer(
    tokenizer=tokenizer,
    min_df=1,
    sublinear_tf=True    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음
)

# 소개글 받아서 유사도 score 추출 --> column화
def get_similarity(df,intro, vectorize):
    intro_df = df[['unique_id','title','intro']]
    scale_df = make_scale(df)
    df = pd.concat([intro_df,scale_df], axis=1)
    model = vectorize.fit(intro_df.intro)
    X = model.transform(intro_df.intro)
    vector = model.transform(intro)
    # 유사도
    cosine_similar = linear_kernel(vector,X).flatten()
    cosine_similar = cosine_similar.round(2)
    df['target_similarity'] = cosine_similar
    
    return df    

In [7]:
# 유사도 기준 내림차순
# Test : 네이버 웹툰
webtoon = "연놈"
intro = find_intro(info_naver, webtoon)
df = get_similarity(info_naver, intro, vectorize=vectorize)
df.sort_values(by="target_similarity", ascending=False).head(5)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64.
  warnings.warn(msg, DataConversionWarning)


,unique_id,title,intro,mean_like,total_comments,mean_score,target_similarity
225,667573,연놈,"중2 여름, 가지 말라는 말도 좋아했다는 말도 하지 못한 채 보낸 그녀가 3년 만에...",0.263979,0.083217,0.988314,1.00
669,713055,호곡,'이제 우리 차례야'조선의 왕 '천군'이 사라지고 3년 뒤...한 소녀가 나타나다.,0.031438,0.001078,0.861436,0.27
612,578106,소나기야,시골마을 사람들과 함께 보내는\r\r\n와루의 여름 이야기..,0.003605,0.004260,0.993322,0.27
671,414609,바람이 머무는 난,어느날 지상 최후의 용과 마주하게 된 한 여자아이의 이야기,0.019902,0.006345,0.993322,0.21
666,705328,환상적인 소년,평범한 소년과 환상적인 소녀의 위험한 동행 이야기.,0.076864,0.002302,1.000000,0.19


**장르 키워드도 같이 intro에 추가해서 돌려야...**

In [5]:
# Total score 기준 내림차순
webtoon = "연애혁명"
intro = find_intro(info_naver, webtoon)
df = get_similarity(info_naver, intro, vectorize=vectorize)
df['total_recommend_score'] = np.sum(df.iloc[:,3:], axis=1)
df.sort_values(by="total_recommend_score", ascending=False).head(5)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64.
  warnings.warn(msg, DataConversionWarning)


,unique_id,title,intro,mean_like,total_comments,mean_score,target_similarity,total_recommend_score
194,570503,연애혁명,"로맨스, 그런 건 우리에게 있을 수가 없어!신개념 개그 로맨스",0.350541,0.454952,0.983306,1.00,2.788798
18,20853,마음의소리,"마사루의 센스를, 이나중의 황당함을 뛰어 넘는다! 솔직 담백 최강의 개그 만화 ! ...",0.071059,1.000000,0.981636,0.09,2.142695
3,680911,무한도전 릴레이툰,무한도전X웹툰작가의 뒤 없이 달리는 릴레이 웹툰이\r\r\n방송과 함께 네이버 웹툰...,1.000000,0.024946,0.919866,0.00,1.944813
197,183559,신의 탑,자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년그리고 그런 소년을 시험하는 탑,0.186566,0.671172,0.991653,0.00,1.849391
216,25455,노블레스,820년간의 긴 수면기. 드디어 새로운 세상에 눈을 뜨다.\r\r\n그리고 새로운 ...,0.093503,0.678079,0.938230,0.00,1.709813


**합산하면 결과가 별로...다...회차 평균 댓글수로 수정을 해봐야 할 것 같다.**

In [81]:
# 다음 웹툰 intro로 네이버 웹툰 유사도 측정
intro = ["거만한 천재 소년과 심약한 은둔 고수와의 만남, 재능과 노력만으론 최고가 될 수 없다!"]
df = get_similarity(info_naver, intro, vectorize=vectorize)
df.sort_values(by="target_similarity", ascending=False).head(5)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64.
  warnings.warn(msg, DataConversionWarning)


,unique_id,title,intro,mean_like,total_comments,mean_score,target_similarity
624,670145,킬더킹,"왕의 재능이란 무엇일까?왕좌의 새로운 주인, 진정한 왕에 걸맞는 재능을 가려내는 게...",0.019374,0.010132,0.948247,0.28
678,675822,대작,천재 신인작가와 그를 쫓는 이들의 숨막히는 이야기,0.070070,0.000911,0.993322,0.23
203,662774,고수,최강의 콤비가 무협의 전설을 다시 쓰다! 천하제일의 고수 강룡. 그리고 수많은 다...,0.212850,0.073468,0.996661,0.23
451,308801,봄이여 오라,"평범한 일상 속, 평범한 만남.\r\r\n그 안에서 엮이는 조금은 특별한 인연.",0.003284,0.006291,0.989983,0.19
532,697681,목욕의 신 (재),신의 손을 가진 남자최고의 목욕관리사가 되기위한 때밀이 배틀이 시작된다.,0.228599,0.000963,0.996661,0.17


---
# Naver + Daum 장르(소개글) 유사도 분석
## Intro + 장르
- Naver Daum 통합
- 장르 --> 한글화
- `장르 + intro` tokenize

In [10]:
info_naver = pd.read_csv("data_Juno/webtoon_naver_info_by_genres.csv")
info_naver.head(1)

,unique_id,title,author,total_score,main_link,image_link,update_date,genre,pay,total_like,intro,total_episodes,mean_score,total_comments,mean_like
0,651673,유미의 세포들,이동건,9.97,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2018.08.31,episode/daily/comic,0,889480,유미와 그녀의 우당탕탕 세포들 이야기.,336,9.97,3171813,36912.94


In [11]:
info_naver[info_naver.title == "호랑이형님"]

,unique_id,title,author,total_score,main_link,image_link,update_date,genre,pay,total_like,intro,total_episodes,mean_score,total_comments,mean_like
209,650305,호랑이형님,이상규,9.98,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2018.08.31,story/fantasy/action/historical,0,470311,신비한 힘을 가진 아이를 이용하여 세상을 지배하려는 반인반수 흰눈썹! 그리고 얼떨결...,173,9.98,453870,16884.16


In [12]:
info_daum = pd.read_csv("./D_Webtoon_4_new_info_Bright.csv",encoding='cp949')
info_daum.tail(1)

,index,id,nickname,title,finishYn,ageGrade,restYn,dateCreated,isNew,averageScore,artist,latest_upload,last_chapter,img_url,intro,genres,categories
765,765,1222,gochoo,여기 고수 추가요!,Y,0,N,20150703094553,0,3.404768,흑석/흑석,2.015123e+13,26.0,http://i1.cartoon.daumcdn.net/svc/image/U03/ca...,고수는 가까이에 있다! 일상에서 그들을 만나서 깨우쳐라!,코믹,도사/모험/무림/잠재능력/스토리


In [13]:
# naver webtoon의 장르 한글화
dic = {'episode':"에피소드", 'omnibus':"옴니버스", 'story':"스토리", 'daily':"일상", 'comic':"코믹", 'fantasy':"판타지", 'action':"액션",
       'drama':"드라마", 'pure':"순정", 'sensibility':"감성", 'thrill':"스릴러", 'historical':"시대극", 'sports':"스포츠"}

info_naver['genres_kor'] = info_naver['genre'].map(lambda x: " ".join([dic[genre] for genre in x.split("/")]))
info_naver.tail(1)

,unique_id,title,author,total_score,main_link,image_link,update_date,genre,pay,total_like,intro,total_episodes,mean_score,total_comments,mean_like,genres_kor
744,347685,슈퍼트리오 시즌2,황미나,7.36,https://comic.naver.com/webtoon/list.nhn?title...,https://shared-comic.pstatic.net/thumb/webtoon...,2012.06.04,story/comic/drama,1,29,대한민국 명작 만화 초대선.\r\r\n살아 있는 전설 황미나 작가의 명작 “슈퍼트리...,6,7.35,1374,3.4,스토리 코믹 드라마


In [14]:
# daum webtoon 카테고리 "/"제거
# NaN값 " " 대체
info_daum['categories'] = info_daum['categories'].map(lambda x : " " if x is np.nan else x )

info_daum['categories'] = info_daum['categories'].map(lambda x: " ".join(x.split("/")))
info_daum.tail(1)

,index,id,nickname,title,finishYn,ageGrade,restYn,dateCreated,isNew,averageScore,artist,latest_upload,last_chapter,img_url,intro,genres,categories
765,765,1222,gochoo,여기 고수 추가요!,Y,0,N,20150703094553,0,3.404768,흑석/흑석,2.015123e+13,26.0,http://i1.cartoon.daumcdn.net/svc/image/U03/ca...,고수는 가까이에 있다! 일상에서 그들을 만나서 깨우쳐라!,코믹,도사 모험 무림 잠재능력 스토리


In [15]:
# intro + 장르 = "text"

# naver
info_naver['text'] = info_naver['intro'] + " " + info_naver['genres_kor']

# daum
info_daum['text'] = info_daum['intro'] + " " + info_daum['genres'] + " " +info_daum['categories']

In [16]:
info_daum[info_daum.title == "여기 고수 추가요!"].text.values

array(['고수는 가까이에 있다! 일상에서 그들을 만나서 깨우쳐라! 코믹 도사 모험 무림 잠재능력 스토리'],
      dtype=object)

**함수 구성**

In [17]:
from konlpy.corpus import kolaw
from konlpy.tag import *
import konlpy

openkoreantext = Okt()

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity, euclidean_distances

In [18]:
# 통합 dataframe 구성
info_naver['platform'] = "naver"
info_daum['platform'] = "daum"
df = pd.concat([info_naver[['title','text','platform']], info_daum[['title','text','platform']]], axis=0)
df.reset_index(drop=True, inplace=True)
df.tail(5)

,title,text,platform
1506,군대라면,내가 겪은 군대는 이렇다! 실제 사연을 바탕으로 각색된 군피소드 웹툰 코믹 병맛 군...,daum
1507,프리스타일 풋볼,"한국형 축구게임의 새로운 기준, 이제 웹툰으로 만나다! 스포츠 축구 게임 병맛 스토리",daum
1508,죽음이 본다,"1997년 서울, 어둠 속에서 한 가족을 지켜보는 눈이 있다. 미스터리 생활고 가족...",daum
1509,광해이야기2,2011년 추석 특집 3부작 단편 광해이야기 그 두번째 공포 괴담 귀신 경험담 스토리,daum
1510,여기 고수 추가요!,고수는 가까이에 있다! 일상에서 그들을 만나서 깨우쳐라! 코믹 도사 모험 무림 잠재...,daum


In [19]:
idx = df[df.text.isnull()].index
df.drop(index=idx, axis=0 ,inplace=True)
df.reset_index(drop=True, inplace=True)

In [20]:
def tokenizer(raw, pos=["Noun","Alpha","Verb","Number"], stopword=[]):
    return [
        word for word, tag in openkoreantext.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

vectorize = TfidfVectorizer(
    tokenizer=tokenizer,
    min_df=1,
    sublinear_tf=True    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음
)
def find_text(df, webtoon_name):
    return df[df.title == webtoon_name].text # 여기서 나온 intro가 get_similarity의 parameter로 간다.

vectorize = TfidfVectorizer(
    tokenizer=tokenizer,
    min_df=1,
    sublinear_tf=True    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음
)

# 소개글 받아서 유사도 score 추출 --> column화
def get_similarity(df,text, vectorize):
    model = vectorize.fit(df.text)
    X = model.transform(df.text)
    vector = model.transform(text)
    # 유사도
#     similarity = linear_kernel(vector,X).flatten()
    similarity = cosine_similarity(vector,X).flatten()
    similarity = similarity.round(2)
    df['target_similarity'] = similarity
    
    return df 

In [21]:
model = vectorize.fit(df.text)

In [22]:
import pickle

In [23]:
with open("4_models/genre_model.pkl", "wb") as f:
    pickle.dump(model,f)

In [25]:
X = vectorize.fit_transform(df.text)

In [28]:
with open("4_models/genre_vectorX.pkl", "wb") as f:
    pickle.dump(X,f)

In [32]:
with open("4_models/genre_vectorX.pkl", "rb") as f:
    a = pickle.load(f)

In [33]:
a.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [26]:
webtoon = "고수"
text = find_text(df, webtoon)
vec = model.transform(text)
similarity = cosine_similarity(vec,X).flatten()
similarity = similarity.round(2)
df['target_similarity'] = similarity
df.sort_values(by="target_similarity", ascending=False).head(5)

,title,text,platform,target_similarity
203,고수,최강의 콤비가 무협의 전설을 다시 쓰다! 천하제일의 고수 강룡. 그리고 수많은 다...,naver,1.00
995,트리니티 원더,무협세계 최강고수와 판타지 세계 천재 마법사가 만난다면? 브레이커 두 작가의 신작!...,daum,0.28
1202,오랑캐,무림 고수를 사칭하는 무림 초짜의 이야기 무협 무림 영웅 무장 성장드라마 스토리,daum,0.23
304,전설의 레전드,전설을 만드는 전설의 전설 스토리 코믹 액션,naver,0.22
292,버닝헬,"윤인완, 양경일 콤비가 다시 뭉쳤다! \r\r\n지상 최악의 섬에서 펼쳐지는 두 ...",naver,0.18


In [18]:
webtoon = "고수"
text = find_text(df, webtoon)
df = get_similarity(df, text, vectorize=vectorize)
df.sort_values(by="target_similarity", ascending=False).head(5)

,title,text,platform,target_similarity
203,고수,최강의 콤비가 무협의 전설을 다시 쓰다! 천하제일의 고수 강룡. 그리고 수많은 다...,naver,1.00
995,트리니티 원더,무협세계 최강고수와 판타지 세계 천재 마법사가 만난다면? 브레이커 두 작가의 신작!...,daum,0.28
1202,오랑캐,무림 고수를 사칭하는 무림 초짜의 이야기 무협 무림 영웅 무장 성장드라마 스토리,daum,0.23
304,전설의 레전드,전설을 만드는 전설의 전설 스토리 코믹 액션,naver,0.22
292,버닝헬,"윤인완, 양경일 콤비가 다시 뭉쳤다! \r\r\n지상 최악의 섬에서 펼쳐지는 두 ...",naver,0.18


In [19]:
webtoon = "트리니티 원더"
text = find_text(df, webtoon)
df = get_similarity(df, text, vectorize=vectorize)
df.sort_values(by="target_similarity", ascending=False).head(5)

,title,text,platform,target_similarity
995,트리니티 원더,무협세계 최강고수와 판타지 세계 천재 마법사가 만난다면? 브레이커 두 작가의 신작!...,daum,1.00
1202,오랑캐,무림 고수를 사칭하는 무림 초짜의 이야기 무협 무림 영웅 무장 성장드라마 스토리,daum,0.34
203,고수,최강의 콤비가 무협의 전설을 다시 쓰다! 천하제일의 고수 강룡. 그리고 수많은 다...,naver,0.28
356,마법스크롤 상인 지오,"마법사인 듯, 마법사 아닌마법사 같은 너! 오지오! 스토리 판타지",naver,0.27
1485,여기 고수 추가요!,고수는 가까이에 있다! 일상에서 그들을 만나서 깨우쳐라! 코믹 도사 모험 무림 잠재...,daum,0.24


In [17]:
# 새로생긴 웹툰으로 테스트
text = ["<노블레스> 손제호 작가와 <소녀더와일즈> 제나 작가가 만났다! 고양이 몸에 깃든 각성자 카이든과 각성능력을 숨겨온 고등학생 지우 러블리 2인조의 액션 판타지"]
df = get_similarity(df, text, vectorize=vectorize)
df.sort_values(by="target_similarity", ascending=False).head(5)

,title,text,platform,target_similarity
263,어빌리티,조금 편리한 능력일 뿐이라고 생각했다. 그 일이 있기 전까지...\r\r\n'노블레...,naver,0.40
294,소녀더와일즈,", 제나(김혜진) 작가와 HUN 작가의 만남.\r\r\n그들이 들려주는, 다소 과...",naver,0.23
1156,고양이 장례식,"헤어진 그와 그녀, 고양이 장례식에서 만나다. 홍작가의 7부작 단편극장 순정 연애 ...",daum,0.20
1135,고.흔.연,고양이를 통해 만난 흔해빠질 수 있는 그와 그녀의 인연 순정 연애 고등학생 고양이 ...,daum,0.18
1443,가디언스,숨겨진 능력을 깨우친 히어로의 강력하고 박진감 넘치는 첫 대결! 판타지 영웅 능력자...,daum,0.17
